In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Wildlife Trade Analysis") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()


In [ ]:
file_path = '/content/drive/MyDrive/wildlife_trade.csv'


In [ ]:
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.show(5)



+----+----+--------------------+-----+-------------+------------+----------+---------+
|Year|App.|               Taxon|Class|        Order|      Family|     Genus|     Term|
+----+----+--------------------+-----+-------------+------------+----------+---------+
|2016|   I|      Aquila heliaca| Aves|Falconiformes|Accipitridae|    Aquila|   bodies|
|2016|   I|      Aquila heliaca| Aves|Falconiformes|Accipitridae|    Aquila|   bodies|
|2016|   I|Haliaeetus albicilla| Aves|Falconiformes|Accipitridae|Haliaeetus| feathers|
|2016|   I|Haliaeetus albicilla| Aves|Falconiformes|Accipitridae|Haliaeetus|specimens|
|2016|   I|Haliaeetus albicilla| Aves|Falconiformes|Accipitridae|Haliaeetus|specimens|
+----+----+--------------------+-----+-------------+------------+----------+---------+
only showing top 5 rows



In [ ]:
df.createOrReplaceTempView("wildlife_trade")


**Analyzing the dataset to identify the most frequently traded animal class.**

In [ ]:

spark.sql("""
    SELECT Class, COUNT(*) as count
    FROM wildlife_trade
    WHERE Class IS NOT NULL
    GROUP BY Class
    ORDER BY count DESC
    LIMIT 1
""").show()



+--------+-----+
|   Class|count|
+--------+-----+
|Reptilia|18430|
+--------+-----+



**Listing all the items (Term) traded that are associated with Mammals**

In [ ]:
spark.sql("SELECT DISTINCT Class FROM wildlife_trade").show()

+--------------+
|         Class|
+--------------+
|          Aves|
|      Bivalvia|
|      Amphibia|
|       Insecta|
|      Mammalia|
|Elasmobranchii|
|     Arachnida|
|    Gastropoda|
|      Reptilia|
|   Actinopteri|
|      Anthozoa|
|     Dipneusti|
|  Hirudinoidea|
|   Coelacanthi|
|      Hydrozoa|
| Holothuroidea|
|          NULL|
+--------------+



In [ ]:
items_traded_mammals = spark.sql("""
    SELECT DISTINCT Term
    FROM wildlife_trade
    WHERE Class = 'Mammalia'
""")

items_traded_mammals.show()


+--------------------+
|                Term|
+--------------------+
|                gall|
|                meat|
| fur product (small)|
|              fibres|
|              scales|
|         bone pieces|
|            garments|
|            medicine|
|              bodies|
|            carvings|
|               horns|
|leather products ...|
|               claws|
|              skulls|
|             extract|
|               tails|
|                hair|
|           specimens|
|         skin pieces|
|         horn pieces|
+--------------------+
only showing top 20 rows



**Identifying all species classified under CITES Appendix II.**

In [ ]:
spark.sql("""
    SELECT DISTINCT Taxon
    FROM wildlife_trade
    WHERE `App.` = 'II'
""").show()



+--------------------+
|               Taxon|
+--------------------+
|   Aquila chrysaetos|
|     Buteo polyosoma|
|Ferocactus chrysa...|
|Geohintonia mexicana|
|Melocactus broadwayi|
|        Parodia spp.|
|     Cebus albifrons|
|Zygosicyos tripar...|
|Euphorbia longifolia|
|Cyphastrea microp...|
|   Chalcopsitta atra|
|Myrmecophaga trid...|
|    Nepenthes hamata|
|  Acianthera calypso|
|  Aerides flabellata|
|Bulbophyllum elon...|
|Bulbophyllum odor...|
|Cephalantheropsis...|
|Dendrobium nathan...|
|Dendrobium tanger...|
+--------------------+
only showing top 20 rows



**Exploring trade data for 2017 to identify the top animal taxon.**



In [ ]:
spark.sql("""
    SELECT Taxon, COUNT(*) AS trade_count
    FROM wildlife_trade
    WHERE Year = 2017
    GROUP BY Taxon
    ORDER BY trade_count DESC
    LIMIT 1
""").show()


+--------------------+-----------+
|               Taxon|trade_count|
+--------------------+-----------+
|Alligator mississ...|         18|
+--------------------+-----------+



**Querying wildlife trade data to find the classes of animals whose trade includes teeth.**


In [ ]:
spark.sql("""
    SELECT DISTINCT Class
    FROM wildlife_trade
    WHERE Term = 'teeth'
""").show()


+--------+
|   Class|
+--------+
|Mammalia|
|Reptilia|
+--------+




 **Identifying animal classes that are traded as live specimens.**

In [ ]:
spark.sql("""
    SELECT DISTINCT Class
    FROM wildlife_trade
    WHERE Term = 'live'
""").show()


+--------------+
|         Class|
+--------------+
|          Aves|
|      Amphibia|
|       Insecta|
|      Mammalia|
|Elasmobranchii|
|     Arachnida|
|    Gastropoda|
|      Reptilia|
|   Actinopteri|
|      Anthozoa|
|     Dipneusti|
|  Hirudinoidea|
|      Hydrozoa|
|      Bivalvia|
|          NULL|
+--------------+



   **Identifying animal classes involved in the trade of carvings.**

In [ ]:
spark.sql("""
    SELECT DISTINCT Class
    FROM wildlife_trade
    WHERE Term = 'carvings'
""").show()


+----------+
|     Class|
+----------+
|      Aves|
|  Bivalvia|
|  Mammalia|
|Gastropoda|
|  Reptilia|
|  Anthozoa|
|  Hydrozoa|
|      NULL|
+----------+

